In [ ]:
pip install numpy==1.22

In [ ]:
pip install pandas

In [ ]:
pip install shap

In [ ]:
pip install seaborn

In [ ]:
pip install optuna

In [ ]:
pip install xgboost

In [ ]:
pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np
import copy
import shap
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
from optuna.samplers import TPESampler
from optuna.trial import Trial
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, KFold, LeaveOneOut, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.utils import shuffle
from sklearn.base import clone
from sklearn import preprocessing

C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
filename = './머신러닝데이터.xlsx'

In [ ]:
# Imputation한 데이터를 불러온다
scoring_transition_1 = pd.read_excel(filename, sheet_name='scoring_result_merged', engine='openpyxl')
scoring_transition_0 = pd.read_excel(filename, sheet_name='scoring_origin', engine='openpyxl')
task_completion_time = pd.read_excel(filename, sheet_name='task_completion_time_mean_imput', engine='openpyxl')
sms_reply = pd.read_excel(filename, sheet_name='SMS_reply', engine='openpyxl')
weather_searching_site_region = pd.read_excel(filename, sheet_name='weather_searching_사이트주소', engine='openpyxl')

# 모든 Feature List
sms_reply, weather_searching_사이트주소는 feature 리스트가 같습니다

In [ ]:
# scoring = ['Routine_Screen_Unlock_Pattern', 'Routine_Phone_Register', 'Routine_Phone_Receive', 'Routine_SMS_Reply', 'Routine_Camera', 
# 'Routine_Location_Searching', 'Routine_Weather_Searching', 'Routine_Transfer', 'Routine_Location_Switching', 'Routine_Weather_Switching', 
# 'Transition_Phone_Register', 'Transition_Phone_Receive', 'Transition_SMS_Reply', 'Transition_Camera', 'Transition_Location_Searching', 
# 'Transition_Weather_Searching', 'Transition_Transfer', 'Transition_Location_Switching', 'Transition_Weather_Switching', 'Result_Phone_Register', 
# 'Result_Phone_Receive', 'Result_SMS_Reply', 'Result_Camera', 'Result_Location_Searching', 'Result_Weather_Searching', 'Result_Transfer', 
# 'Result_Location_Switching', 'Result_Weather_Switching', 'routine_sum_trial', 'transition_sum_trial', 'result_sum_trial', 'all_sum_trial']

# task = ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 'phone_register_total_time_nt', 
# 'phone_register_screen_unlocking_time', 'phone_register_sms_start_time', 'phone_register_instruction_check_time', 'phone_register_total_time', 
# 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_receive_noti response', 'phone_receive_total_time_nt', 'phone_receive_total_time', 
# 'phone_receive_missing_call_time_1', 'phone_receive_missing_call_time_2', 'sms_reply_noti_response_time', 'sms_reply_total_time_nt', 'sms_reply_compeltion_time', 
# 'sms_reply_total_time', 'camera_noti_response', 'camera_total_time_nt', 'camera_instruction_check_time', 'camera_total_time', 'camera_taken_time', 
# 'camera_gallery_delete_time', 'transfer_noti_response_time', 'transfer_total_time_nt', 'transfer_instruction_check_time', 'transfer_total_time', 
# 'transfer_usage_time', 'transfer_share_time', 'weather_searching_noti_response', 'weather_searching_total_time_nt', 'weather_searching_instruction_check_time', 
# 'weather_searching_total_time', 'weather_information_searching_time', 'weather_information_sharing_time', 'weather_switching_notification_response_time', 
# 'weather_switching_total_time_nt', 'weather_switching_instruction_check_time', 'weather_switching_total_time', 
# 'weather_switching_first_searching_keyword_typing_time', 'weather_switching_second_searching_keyword_typing_time', 
# 'weather_switching_information_searching_time', 'weather_switching_information_sharing_texting_time', 'location_searching_noti_response_time', 
# 'location_searching_total_time_nt', 'location_searching_instruction_check_time', 'location_searching_total_time', 'location_searching_food_typing_time', 
# 'location_searching_foodtyping_mapfinding_time', 'location_searching_mapfinding_time', 'location_searching_route_sharing_texting_time', 
# 'location_switching_notification_response', 'location_switching_total_time_nt', 'location_switching_instruction_check', 'location_switching_total_time', 
# 'location_switching_first_map_typing_time', 'location_switching_second_map_typing_time', 'location_switching_map_finding_time', 
# 'location_switching_map_route_sharing_time']

# sms_and_ws = ['average intercharacter time', 'std intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
# 'total_time (S)', 'total_count', 'IS (C+IF+INF+F)', 'backspace (F)', 'C', 'IF', 'INF', 'Shift', 'ISø (IS+S)', 'T (total_count-F)', 'WPS', 'WPM', 'AdjWPS', 
# 'AdjWPM', 'CPS', 'KSPS', 'GPS', 'KSPC', 'GPC', 'MSD', 'COER', 'UER', 'TER', 'CE', 'PC', 'UB', 'WB', 'CPC']

In [ ]:
# BAC column만 가져온다(Label)
y = scoring_transition_1['BAC']

# Participant, BAC column을 제거한다
scoring_transition_1 = scoring_transition_1.drop(['Participant', 'BAC'], axis=1)
scoring_transition_0 = scoring_transition_0.drop(['Participant', 'BAC'], axis=1)
task_completion_time = task_completion_time.drop(['Participant', 'BAC'], axis=1)
sms_reply = sms_reply.drop(['Participant', 'BAC'], axis=1)
weather_searching_site_region = weather_searching_site_region.drop(['Participant', 'BAC'], axis=1)

In [ ]:
# 라벨 3개용 features
# 각 데이터별로 필요한 column들만 뽑는다
scoring_transition_1_columns = []

scoring_transition_0_columns = ['Routine_Screen_Unlock_Pattern', 'Transition_Phone_Register', 'Transition_Phone_Receive', 'Transition_SMS_Reply', 
                                'Transition_Camera', 'Transition_Weather_Searching', 'Transition_Transfer', 'transition_sum_trial', 'Result_Phone_Register', 'Result_Phone_Receive', 'Result_SMS_Reply',
                                'Result_Camera', 'Result_Weather_Searching', 'Result_Transfer', 'result_sum_trial']

task_completion_time_columns = ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 
'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 
'phone_receive_total_time','phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 
'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 
'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time']

sms_reply_columns = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'GPS', 'COER', 'UER', 'TER', 'UB', 'WB']

weather_searching_site_region_columns = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'COER', 'UB', 'WB']

In [ ]:
# 라벨을 2개용 features
scoring_transition_1_columns_two_label = []

scoring_transition_0_columns_two_label = ['Routine_Screen_Unlock_Pattern', 'Transition_Phone_Register', 'Transition_Phone_Receive', 'Transition_SMS_Reply', 
                                        'Transition_Camera', 'Transition_Weather_Searching', 'Transition_Transfer', 'transition_sum_trial', 'Result_Phone_Register', 'Result_Phone_Receive', 'Result_SMS_Reply',
                                        'Result_Camera', 'Result_Weather_Searching', 'Result_Transfer', 'result_sum_trial']

task_completion_time_columns_two_label = ['average_noti_response_app_start', 'median_noti_response_app_start', 'phone_register_noti_response', 
'phone_register_screen_unlocking_time', 'phone_register_phone_number_register_time', 'phone_register_call_time', 'phone_register_total_time', 
'phone_receive_total_time','phone_receive_missing_call_time_2', 'sms_reply_compeltion_time', 'sms_reply_total_time', 
'camera_taken_time', 'camera_gallery_delete_time', 'camera_total_time', 'transfer_usage_time', 'transfer_share_time', 'transfer_total_time', 
'weather_information_searching_time', 'weather_information_sharing_time', 'weather_searching_total_time']

sms_reply_columns_two_label = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'GPS', 'COER', 'UER', 'TER', 'UB', 'WB']

weather_searching_site_region_columns_two_label = ['average intercharacter time', 'median intercharacter time', 'min intercharacter time', 'max intercharacter time', 
'CPS', 'KSPS', 'COER', 'UB', 'WB']

In [ ]:
# 위에서 정한 feature들로 각각의 데이터 프레임을 생성한다
scoring_transition_1_df = scoring_transition_1[scoring_transition_1_columns]
scoring_transition_0_df = scoring_transition_0[scoring_transition_0_columns]
task_completion_time_df = task_completion_time[task_completion_time_columns]
sms_reply_df = sms_reply[sms_reply_columns]
weather_searching_site_region_df = weather_searching_site_region[weather_searching_site_region_columns]

scoring_transition_1_df_two_label = scoring_transition_1[scoring_transition_1_columns_two_label]
scoring_transition_0_df_two_label = scoring_transition_0[scoring_transition_0_columns_two_label]
task_completion_time_df_two_label = task_completion_time[task_completion_time_columns_two_label]
sms_reply_df_two_label = sms_reply[sms_reply_columns_two_label]
weather_searching_site_region_df_two_label = weather_searching_site_region[weather_searching_site_region_columns_two_label]

# feature명 수정하기
위에서 적은 feature명을 그대로 복사해서 가져온 다음 바꾸려는 feature만 수정하면 됩니다

In [ ]:
# 라벨 3개용 features의 feature명을 수정한다!
scoring_transition_1_columns = []

scoring_transition_0_columns = ['screen_unlock_trial', 'phone_register_transition', 'phone_receive_transition', 'SMS_reply_transition', 
'camera_transition', 'info_sear_transition', 'banking_transition', 'total_task_sum_transition', 'phone_register_accuracy', 'phone_receive_accuracy', 'SMS_reply_accuracy',
                                        'camera_accuracy', 'weather_searching_accuracy', 'transfer_accuracy', 'total_task_sum_accuracy']

task_completion_time_columns = ['average_noti_response_time', 'median_noti_response_time', 'screen_on_after_noti_response', 
'screen_Unlock_Time', 'phone_number_register_time', 'phone_call_time', 'phone_call_register_task_total_time', 
'phone_receive_task_total_time','phone_receive_time', 'sms_reply_compeltion_time', 'sms_reply_task_total_time', 
'camera_taken_time', 'image_delete_time', 'camera_task_total_time', 'banking_usage_time', 'transfer_share_time', 'banking_task_total_time', 
'info_searching_time', 'info_sharing_time', 'info_search_share_task_total_time']

sms_reply_columns = ['average_intercharacter_time', 'median_intercharacter_time', 'min_intercharacter_time', 'max_intercharacter_time', 
'CPS', 'KSPS', 'GPS', 'COER', 'UER', 'TER', 'UB', 'WB']

weather_searching_site_region_columns = ['average_intercharacter_time', 'median_intercharacter_time', 'min_intercharacter_time', 'max_intercharacter_time', 
'CPS', 'KSPS', 'COER', 'UB', 'WB']

scoring_transition_1_df.columns = scoring_transition_1_columns
scoring_transition_0_df.columns = scoring_transition_0_columns
task_completion_time_df.columns = task_completion_time_columns
sms_reply_df.columns = sms_reply_columns
weather_searching_site_region_df.columns = weather_searching_site_region_columns

In [ ]:
# 라벨을 2개용 features의 feature명을 수정한다!
scoring_transition_1_columns_two_label = []

scoring_transition_0_columns_two_label = ['screen_unlock_trial', 'phone_register_transition', 'phone_receive_transition', 'SMS_reply_transition', 
'camera_transition', 'info_sear_transition', 'banking_transition', 'total_task_sum_transition', 'phone_register_accuracy', 'phone_receive_accuracy', 'SMS_reply_accuracy',
                                        'camera_accuracy', 'weather_searching_accuracy', 'transfer_accuracy', 'total_task_sum_accuracy']

task_completion_time_columns_two_label = ['average_noti_response_time', 'median_noti_response_time', 'screen_on_after_noti_response', 
'screen_Unlock_Time', 'phone_number_register_time', 'phone_call_time', 'phone_call_register_task_total_time', 
'phone_receive_task_total_time','phone_receive_time', 'sms_reply_compeltion_time', 'sms_reply_task_total_time', 
'camera_taken_time', 'image_delete_time', 'camera_task_total_time', 'banking_usage_time', 'transfer_share_time', 'banking_task_total_time', 
'info_searching_time', 'info_sharing_time', 'info_search_share_task_total_time']

sms_reply_columns_two_label = ['average_intercharacter_time', 'median_intercharacter_time', 'min_intercharacter_time', 'max_intercharacter_time', 
'CPS', 'KSPS', 'GPS', 'COER', 'UER', 'TER', 'UB', 'WB']

weather_searching_site_region_columns_two_label = ['average_intercharacter_time', 'median_intercharacter_time', 'min_intercharacter_time', 'max_intercharacter_time', 
'CPS', 'KSPS', 'COER', 'UB', 'WB']

scoring_transition_1_df_two_label.columns = scoring_transition_1_columns_two_label
scoring_transition_0_df_two_label.columns = scoring_transition_0_columns_two_label
task_completion_time_df_two_label.columns = task_completion_time_columns_two_label
sms_reply_df_two_label.columns = sms_reply_columns_two_label
weather_searching_site_region_df_two_label.columns = weather_searching_site_region_columns_two_label

In [ ]:
# sms reply와 weather searching 사이트 주소의 feature명이 똑같으므로 feature명을 수정한다
sms_reply_df_columns = ['sms_' + col_name for col_name in sms_reply_df.columns] # sms reply feature 이름 앞에 sms_를 붙인다
weather_searching_site_region_df_columns = ['ws_' + col_name for col_name in weather_searching_site_region_df.columns] # ws feature 이름 앞에 ws_를 붙인다

sms_reply_df.columns = sms_reply_df_columns # 수정된 sms reply feature명을 적용한다
weather_searching_site_region_df.columns = weather_searching_site_region_df_columns # 수정된 ws feature명을 적용한다

# 위와 동일하게 작동한다
sms_reply_df_two_label_columns = ['sms_' + col_name for col_name in sms_reply_df_two_label.columns]
weather_searching_site_region_df_two_label_columns = ['ws_' + col_name for col_name in weather_searching_site_region_df_two_label.columns]

sms_reply_df_two_label.columns = sms_reply_df_two_label_columns
weather_searching_site_region_df_two_label.columns = weather_searching_site_region_df_two_label_columns

### merged_df : Label 3개인 데이터 집합
### merged_df_two_label : Label 2개인 데이터 집합

In [ ]:
# 필요한 데이터 프레임들을 합친다
merged_df_list = [scoring_transition_1_df, scoring_transition_0_df, task_completion_time_df, sms_reply_df, weather_searching_site_region_df]
merged_df_list_two_label = [scoring_transition_1_df_two_label, scoring_transition_0_df_two_label, task_completion_time_df_two_label, 
sms_reply_df_two_label, weather_searching_site_region_df_two_label]

merged_df = pd.concat(merged_df_list, axis=1)
merged_df_two_label = pd.concat(merged_df_list_two_label, axis=1)

In [ ]:
# Categorical 데이터에 대해서 인코딩한다
# 어떤 데이터가 Categorical 데이터인가?

# 라벨에 대해서 인코딩한다
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(y)

### N : 3개 라벨에 대한 feature 개수 / M : 2개 라벨에 대한 feature 개수
merged_df.shape = (360, N), y.shape = (360, 1), 라벨 수: 3 </br>
merged_df_two_label.shape = (360, M), changed_label.shape = (360, 1), 라벨 수: 2 </br>
changed_df.shape = (240, M), changed_y.shape = (240, 1), 라벨 수: 2 </br>

In [ ]:
# 라벨에 대해서 인코딩한다(0 -> 0, 0.06 -> 1, 0.09 -> 2)
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(y)

# 3개의 라벨을 2개의 라벨로 바꾼다
changed_label = copy.deepcopy(y)

for i in range(len(changed_label)):
    if changed_label[i] == 1:
        changed_label[i] = 0

for i in range(len(changed_label)):
    if changed_label[i] == 2:
        changed_label[i] = 1

# 라벨 0과 라벨 0.06의 평균을 구해서 하나의 라벨로 만든다
temp_y = pd.Series(y, name='Label')
temp_df = pd.concat([temp_y, merged_df_two_label], axis=1)

changed_df = pd.DataFrame(columns=temp_df.columns)

num = 0
for row_idx in range(0, len(temp_df), 9):
    for i in range(3):
        changed_df.loc[num] = (temp_df.loc[row_idx + i] + temp_df.loc[row_idx + i + 3])/2
        num += 1
    for i in range(3):
        changed_df.loc[num] = temp_df.loc[row_idx + 6 + i]
        num += 1

changed_y = changed_df['Label']
changed_df = changed_df.drop(['Label'], axis=1)
changed_y = encoder.fit_transform(changed_y)

# 3 Labels, 2 Labels(just change), 2 Labels(average)에 대한 데이터
label_processing = ['3 Labels', '2 Labels(just change)', '2 Labels(average)']
data_list = [(merged_df, y), (merged_df_two_label, changed_label), (changed_df, changed_y)]

In [ ]:
scaler_list = {
    'minmax': MinMaxScaler(),
    'maxabs': MaxAbsScaler(),
    'standard': StandardScaler(),
    'robust': RobustScaler()
}

label_list = {
    'three_label': data_list[0],
    'two_label_changed': data_list[1],
    'two_label_average': data_list[2]
}

# scaler_name과 label_name을 인자로 받아서 해당하는 X, y를 반환한다
def getXy(scaler_name, label_name):
    if scaler_name not in scaler_list.keys() and scaler_name != 'no_scale':
        raise ValueError("scaler_name must be 'minmax' or 'maxabs' or 'standard' or 'robust' or 'no_scale!")

    if label_name not in label_list.keys():
        raise ValueError("label_name must be 'three_label' or 'two_label_changed' or 'two_label_average'!")

    X, y = label_list[label_name]

    if scaler_name != 'no_scale':
        scaler = scaler_list[scaler_name]
        df = X.copy()

        df[df.columns] = scaler.fit_transform(df[df.columns])

        X = df
        X[X.columns] = scaler.fit_transform(X[X.columns])
    
    return (X, y)

# 어떤 label의 데이터로 돌릴 것인지, 어떤 scaler를 적용할 것인지, inner loop에서 어떤 cv 방법을 적용할 것인지, Stratified K-fold의 K를 정한다

### scaler_name = 'minmax' or 'maxabs' or 'standard' or 'robust' or 'no_scale'
### label_name = 'three_label' or 'two_label_changed' or 'two_label_average'
### cv_method = 'loso' or 'k_fold'
### n_fold = 39(default)

In [ ]:
scaler_name = 'robust'
label_name = 'two_label_changed'
cv_method = 'k_fold'
n_fold = 13

X, y = getXy(scaler_name=scaler_name, label_name=label_name)

# Optuna의 반복 수를 설정한다

In [ ]:
n_trials = 200

In [ ]:
def NestedCVwithOptuna(objective, clf, clf_name):
    metric_df = pd.DataFrame(columns=['model', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc_score'])

    # 클래스가 몇 개인지 확인한다
    class_nums = len(set(y))
    print('class_nums : ', class_nums)
    n_splits = 40
    kfold = KFold(n_splits=n_splits)

    mean_accuracy = 0
    mean_precision = 0
    mean_recall = 0
    mean_f1 = 0
    mean_roc_auc = 0
    
    list_shap_values = list()
    list_test_sets = list()

    for num, (train_idx, val_idx) in enumerate(kfold.split(X)):
        X_train, X_val = X.iloc[train_idx, :], X.iloc[val_idx, :]
        y_train, y_val = y[train_idx], y[val_idx]
        
        classifier = clone(clf)
        # Bayesian Optimization을 진행한다
        sampler = TPESampler()

        study = optuna.create_study(direction='maximize', sampler=sampler, study_name=f'{clf_name} Study')
        study.optimize(objective(X_train, y_train), n_trials=n_trials)

        best_params = study.best_params
        print('Optuna Best score : ', study.best_value)
        print('Best parameters : ', best_params)


        if clf_name == 'lightgbm':
            best_clf = LGBMClassifier(**best_params, random_state=42)
        elif clf_name == 'xgboost':
            best_clf = XGBClassifier(**best_params, random_state=42)
        elif clf_name == 'randomforest':
            best_clf = RandomForestClassifier(**best_params, random_state=42)
        elif clf_name == 'gbm':
            best_clf = GradientBoostingClassifier(**best_params, random_state=42)
        
        best_clf.fit(X_train, y_train)

        y_pred = best_clf.predict(X_val)
        y_pred_prob = best_clf.predict_proba(X_val)

        # 개별 performance metric을 구한다
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred, average='macro')
        recall = recall_score(y_val, y_pred, average='macro')
        f1 = f1_score(y_val, y_pred, average='macro')
        if class_nums > 2:
            roc_auc = roc_auc_score(y_val, y_pred_prob, average='macro', multi_class='ovr')
        else:
            roc_auc = roc_auc_score(y_val, y_pred)
        
        # 개별 performance metric을 데이터 프레임으로 저장한다
        metric_df.loc[len(metric_df)] = [f'model_{num+1}', accuracy, precision, recall, f1, roc_auc]

        # 평균 performance metric을 구한다
        mean_accuracy += accuracy / n_splits
        mean_precision += accuracy / n_splits
        mean_recall += recall / n_splits
        mean_f1 += f1 / n_splits
        mean_roc_auc += roc_auc / n_splits

        print('test accuracy : ', accuracy)

        # explaining model
        explainer = shap.TreeExplainer(best_clf)
        shap_values = explainer.shap_values(X_val)
        
        list_shap_values.append(shap_values)
        list_test_sets.append(val_idx)

    print()
    print('mean accuracy : ', mean_accuracy)
    print('mean precision : ', mean_precision)
    print('mean recall : ', mean_recall)
    print('mean f1 : ', mean_f1)
    print('mean roc_auc : ', mean_roc_auc)
    print()

    metric_df.loc[len(metric_df)] = ['mean', mean_accuracy, mean_precision, mean_recall, mean_f1, mean_roc_auc]

    # combining results from all iterations
    if class_nums > 2 or clf_name == 'lightgbm' or clf_name == 'randomforest':
        test_set = list_test_sets[0]
        shap_values = np.array(list_shap_values[0])
        for i in range(1,len(list_test_sets)):
            test_set = np.concatenate((test_set,list_test_sets[i]),axis=0)
            shap_values = np.concatenate((shap_values,np.array(list_shap_values[i])),axis=1)
        #bringing back variable names    
        X_test = pd.DataFrame(X.loc[test_set], columns=X.columns)
    else:
        test_set = list_test_sets[0]
        shap_values = np.array(list_shap_values[0])
        for i in range(1,len(list_test_sets)):
            test_set = np.concatenate((test_set,list_test_sets[i]),axis=0)
            shap_values = np.concatenate((shap_values,np.array(list_shap_values[i])),axis=0)
        #bringing back variable names    
        X_test = pd.DataFrame(X.loc[test_set], columns=X.columns)

    return (shap_values, X_test, metric_df)

## Accuracy가 아닌 다른 metric을 기준으로 Optuna을 하려면 precision ~ roc_auc 주석 중 하나를 풀고 return 해주면 됩니다

In [ ]:
def InnerCrossValidation(clf, X, y, cv_method, n_fold=39):
    class_nums = len(set(y))
    if cv_method == 'loso':
        kfold = KFold(n_splits=39)
        splits = kfold.split(X)
    elif cv_method == 'k_fold':
        stratified_kfold = StratifiedKFold(n_splits=n_fold)
        splits = stratified_kfold.split(X, y)
    else:
        raise ValueError("cv_method must be 'loso' or 'k_fold'!")
    
    train_X_len = len(X)
    if train_X_len % n_fold != 0:
        raise ValueError(f"{train_X_len} / {n_fold} is not Integer!")

    y_true_list, y_pred_list = [], []
    y_pred_prob_list = []
    
    for train_idx, val_idx in splits:
        X_train, X_val = X.iloc[train_idx, :], X.iloc[val_idx, :]
        y_train, y_val = y[train_idx], y[val_idx]
        
        classifier = clone(clf)

        # 모델을 학습한다
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_val)
        y_pred_prob = classifier.predict_proba(X_val)
        
        # 실제, 예측 라벨을 저장한다
        y_true_list.extend(y_val)
        y_pred_list.extend(y_pred)
        y_pred_prob_list.extend(y_pred_prob)
    
    # 여기를 바꾸면 됩니다
    accuracy = accuracy_score(y_true_list, y_pred_list)
    # precision = precision_score(y_true_list, y_pred_list, average='macro')
    # recall = recall_score(y_true_list, y_pred_list, average='macro')
    # f1 = f1_score(y_true_list, y_pred_list, average='macro')
    # if class_nums > 2:
    #     roc_auc = roc_auc_score(y_val, y_pred_prob, average='macro', multi_class='ovr')
    # else:
    #     roc_auc = roc_auc_score(y_val, y_pred)

    # 반환하는 변수를 바꿔줘야 합니다
    return accuracy

# Lightgbm Bayesian

In [ ]:
class LGBMObjective(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __call__(self, trial: Trial):
        lgbm_params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'learning_rate': trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.1),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.1), 
            'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 5.0), # lambda_l1
            'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 5.0), # lambda_l2
            'random_state': 42
        }

        clf = LGBMClassifier(**lgbm_params)

        mean_accuracy = InnerCrossValidation(clf, self.X, self.y, cv_method, n_fold)

        return mean_accuracy

In [ ]:
clf = LGBMClassifier(random_state=42)
shap_values, X_test, metric_df = NestedCVwithOptuna(LGBMObjective, clf, 'lightgbm')

class_nums :  2


[I 2022-08-30 22:35:28,778] A new study created in memory with name: lightgbm Study
[I 2022-08-30 22:35:29,493] Trial 0 finished with value: 0.7977207977207977 and parameters: {'n_estimators': 422, 'learning_rate': 0.06822986456091779, 'num_leaves': 200, 'colsample_bytree': 0.9, 'min_child_samples': 54, 'subsample': 1.0, 'reg_alpha': 1.3161373267607668, 'reg_lambda': 3.955073619690101}. Best is trial 0 with value: 0.7977207977207977.
[I 2022-08-30 22:35:30,471] Trial 1 finished with value: 0.7948717948717948 and parameters: {'n_estimators': 467, 'learning_rate': 0.05223102148906488, 'num_leaves': 72, 'colsample_bytree': 1.0, 'min_child_samples': 27, 'subsample': 0.8, 'reg_alpha': 2.2608064942673547, 'reg_lambda': 3.4025401643893027}. Best is trial 0 with value: 0.7977207977207977.
[I 2022-08-30 22:35:31,003] Trial 2 finished with value: 0.7977207977207977 and parameters: {'n_estimators': 258, 'learning_rate': 0.04564015108664602, 'num_leaves': 152, 'colsample_bytree': 0.9, 'min_child_s

[I 2022-08-30 22:35:48,665] Trial 22 finished with value: 0.7492877492877493 and parameters: {'n_estimators': 379, 'learning_rate': 0.003291253946500633, 'num_leaves': 210, 'colsample_bytree': 0.6, 'min_child_samples': 15, 'subsample': 0.9, 'reg_alpha': 1.4909364149194078, 'reg_lambda': 2.8318248811858755}. Best is trial 17 with value: 0.8205128205128205.
[I 2022-08-30 22:35:49,016] Trial 23 finished with value: 0.7521367521367521 and parameters: {'n_estimators': 102, 'learning_rate': 0.010735367418007024, 'num_leaves': 177, 'colsample_bytree': 0.7, 'min_child_samples': 29, 'subsample': 0.9, 'reg_alpha': 1.7627025765028763, 'reg_lambda': 2.0848528898205987}. Best is trial 17 with value: 0.8205128205128205.
[I 2022-08-30 22:35:50,187] Trial 24 finished with value: 0.7891737891737892 and parameters: {'n_estimators': 298, 'learning_rate': 0.025186163482169607, 'num_leaves': 111, 'colsample_bytree': 0.8, 'min_child_samples': 11, 'subsample': 0.8, 'reg_alpha': 2.2659845797995346, 'reg_lambd

[I 2022-08-30 22:36:11,359] Trial 44 finished with value: 0.7948717948717948 and parameters: {'n_estimators': 467, 'learning_rate': 0.05552667423869112, 'num_leaves': 185, 'colsample_bytree': 0.9, 'min_child_samples': 41, 'subsample': 0.9, 'reg_alpha': 1.8299102106574938, 'reg_lambda': 1.6870062561365093}. Best is trial 17 with value: 0.8205128205128205.
[I 2022-08-30 22:36:11,925] Trial 45 finished with value: 0.7236467236467237 and parameters: {'n_estimators': 406, 'learning_rate': 0.0036528518211545644, 'num_leaves': 95, 'colsample_bytree': 0.8, 'min_child_samples': 69, 'subsample': 1.0, 'reg_alpha': 1.4227387199029575, 'reg_lambda': 2.6285130837816766}. Best is trial 17 with value: 0.8205128205128205.
[I 2022-08-30 22:36:13,209] Trial 46 finished with value: 0.8062678062678063 and parameters: {'n_estimators': 363, 'learning_rate': 0.051207790343828094, 'num_leaves': 215, 'colsample_bytree': 0.9, 'min_child_samples': 18, 'subsample': 0.7, 'reg_alpha': 1.0681401793643361, 'reg_lambda

[I 2022-08-30 22:36:34,096] Trial 66 finished with value: 0.7863247863247863 and parameters: {'n_estimators': 449, 'learning_rate': 0.016152316807829702, 'num_leaves': 172, 'colsample_bytree': 0.8, 'min_child_samples': 21, 'subsample': 0.7, 'reg_alpha': 1.5288183418643626, 'reg_lambda': 1.4459505200903242}. Best is trial 17 with value: 0.8205128205128205.
[I 2022-08-30 22:36:35,567] Trial 67 finished with value: 0.8148148148148148 and parameters: {'n_estimators': 333, 'learning_rate': 0.051726400324363375, 'num_leaves': 189, 'colsample_bytree': 0.9, 'min_child_samples': 13, 'subsample': 0.8, 'reg_alpha': 1.0941768383454427, 'reg_lambda': 4.8737880739314265}. Best is trial 17 with value: 0.8205128205128205.
[I 2022-08-30 22:36:36,658] Trial 68 finished with value: 0.8005698005698005 and parameters: {'n_estimators': 287, 'learning_rate': 0.03886637424622203, 'num_leaves': 221, 'colsample_bytree': 0.7, 'min_child_samples': 13, 'subsample': 0.8, 'reg_alpha': 1.2803970023823814, 'reg_lambda

...

[I 2022-08-30 22:57:44,955] Trial 159 finished with value: 0.7891737891737892 and parameters: {'n_estimators': 473, 'learning_rate': 0.020014280517050325, 'num_leaves': 112, 'colsample_bytree': 0.6, 'min_child_samples': 98, 'subsample': 0.7, 'reg_alpha': 3.3662656938571387, 'reg_lambda': 4.231870886160177}. Best is trial 38 with value: 0.811965811965812.
[I 2022-08-30 22:57:45,426] Trial 160 finished with value: 0.792022792022792 and parameters: {'n_estimators': 455, 'learning_rate': 0.026148752633847316, 'num_leaves': 141, 'colsample_bytree': 0.6, 'min_child_samples': 89, 'subsample': 0.7, 'reg_alpha': 3.6773687678496096, 'reg_lambda': 3.8445491267849285}. Best is trial 38 with value: 0.811965811965812.
[I 2022-08-30 22:57:45,856] Trial 161 finished with value: 0.8091168091168092 and parameters: {'n_estimators': 444, 'learning_rate': 0.027754409408219976, 'num_leaves': 122, 'colsample_bytree': 0.6, 'min_child_samples': 100, 'subsample': 0.7, 'reg_alpha': 3.604639582256849, 'reg_lambda

[I 2022-08-30 22:57:57,548] Trial 181 finished with value: 0.7834757834757835 and parameters: {'n_estimators': 458, 'learning_rate': 0.026584239091529193, 'num_leaves': 142, 'colsample_bytree': 0.6, 'min_child_samples': 98, 'subsample': 0.7, 'reg_alpha': 3.6483471277251316, 'reg_lambda': 4.1531153800337455}. Best is trial 38 with value: 0.811965811965812.
[I 2022-08-30 22:57:58,035] Trial 182 finished with value: 0.8034188034188035 and parameters: {'n_estimators': 439, 'learning_rate': 0.02652210642411347, 'num_leaves': 133, 'colsample_bytree': 0.6, 'min_child_samples': 100, 'subsample': 0.7, 'reg_alpha': 3.6341982335486156, 'reg_lambda': 3.956665550518966}. Best is trial 38 with value: 0.811965811965812.
[I 2022-08-30 22:57:58,519] Trial 183 finished with value: 0.8091168091168092 and parameters: {'n_estimators': 494, 'learning_rate': 0.024589089898696296, 'num_leaves': 126, 'colsample_bytree': 0.6, 'min_child_samples': 100, 'subsample': 0.7, 'reg_alpha': 3.702715820174638, 'reg_lambd

Optuna Best score :  0.811965811965812
Best parameters :  {'n_estimators': 268, 'learning_rate': 0.08220510945419537, 'num_leaves': 155, 'colsample_bytree': 0.8, 'min_child_samples': 100, 'subsample': 0.8, 'reg_alpha': 1.432371528175254, 'reg_lambda': 4.0132350415709945}
test accuracy :  1.0


[I 2022-08-30 22:58:07,021] A new study created in memory with name: lightgbm Study
[I 2022-08-30 22:58:07,336] Trial 0 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 167, 'learning_rate': 0.003320138767563956, 'num_leaves': 198, 'colsample_bytree': 0.9, 'min_child_samples': 61, 'subsample': 0.8, 'reg_alpha': 1.8913944985746944, 'reg_lambda': 2.3536531178769624}. Best is trial 0 with value: 0.6666666666666666.
[I 2022-08-30 22:58:07,879] Trial 1 finished with value: 0.7378917378917379 and parameters: {'n_estimators': 124, 'learning_rate': 0.010355822023119035, 'num_leaves': 52, 'colsample_bytree': 0.8, 'min_child_samples': 10, 'subsample': 0.9, 'reg_alpha': 2.236212559591555, 'reg_lambda': 4.663296163903736}. Best is trial 1 with value: 0.7378917378917379.
[I 2022-08-30 22:58:08,261] Trial 2 finished with value: 0.7891737891737892 and parameters: {'n_estimators': 177, 'learning_rate': 0.03864493659472619, 'num_leaves': 220, 'colsample_bytree': 1.0, 'min_child_

[I 2022-08-30 22:58:24,087] Trial 22 finished with value: 0.7977207977207977 and parameters: {'n_estimators': 499, 'learning_rate': 0.018384249206501776, 'num_leaves': 122, 'colsample_bytree': 0.9, 'min_child_samples': 36, 'subsample': 1.0, 'reg_alpha': 4.254983021084328, 'reg_lambda': 3.3180783786008323}. Best is trial 15 with value: 0.8176638176638177.
[I 2022-08-30 22:58:25,238] Trial 23 finished with value: 0.8034188034188035 and parameters: {'n_estimators': 456, 'learning_rate': 0.013561518960461748, 'num_leaves': 149, 'colsample_bytree': 1.0, 'min_child_samples': 24, 'subsample': 0.9, 'reg_alpha': 4.924178978281575, 'reg_lambda': 2.720393309832253}. Best is trial 15 with value: 0.8176638176638177.
[I 2022-08-30 22:58:26,389] Trial 24 finished with value: 0.8091168091168092 and parameters: {'n_estimators': 416, 'learning_rate': 0.029573085066926363, 'num_leaves': 124, 'colsample_bytree': 0.9, 'min_child_samples': 14, 'subsample': 1.0, 'reg_alpha': 3.6769750363208957, 'reg_lambda':

## 40개의 모델의 개별 performance metric + 평균 performnace metric을 저장한다
## 각 모델별 test set의 feature에 따른 shap value를 저장한다

In [ ]:
if label_name == 'three_label':
    shap_values_df_0 = pd.DataFrame(shap_values[0], columns=X_test.columns)
    shap_values_df_1 = pd.DataFrame(shap_values[1], columns=X_test.columns)
    shap_values_df_2 = pd.DataFrame(shap_values[2], columns=X_test.columns)

    with pd.ExcelWriter(f"./lgbm_result_2_label.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df_0.to_excel(writer, sheet_name="shap_list_0", index=False)
        shap_values_df_1.to_excel(writer, sheet_name="shap_list_1", index=False)
        shap_values_df_2.to_excel(writer, sheet_name="shap_list_2", index=False)

if label_name != 'three_label':
    shap_values_df = pd.DataFrame(shap_values[0], columns=X_test.columns)

    with pd.ExcelWriter(f"./lgbm_result_2_label.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df.to_excel(writer, sheet_name="shap_list", index=False)

## 2 Labels

In [ ]:
if label_name != 'three_label':
    shap.summary_plot(shap_values[0], X_test, show=False)
    plt.savefig('shap_summaryplot_lgbm_2label_class0.png', bbox_inches='tight')

In [ ]:
if label_name != 'three_label':
    shap.summary_plot(shap_values[1], X_test, show=False)
    plt.savefig('shap_summaryplot_lgbm_2label_class1.png', bbox_inches='tight')

In [ ]:
if label_name != 'three_label':
    shap_values = [shap_values[0], shap_values[1]]
    shap.summary_plot(shap_values, X_test, class_names=['BAL 0-0.06', 'BAL 0.06-0.09'], class_inds=[0, 1], show=False)
    plt.savefig('shap_absmeanplot_lgbm_2label.png', bbox_inches='tight')

# Xgboost Bayesian

In [ ]:
class XBGObjective(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __call__(self, trial: Trial):    
        xgb_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'learning_rate': trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 9),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
            'gamma': trial.suggest_float('gamma', 0.0, 5.0, step=0.1),
            'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 5.0), # lambda_l1
            'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 5.0), # lambda_l2
            'random_state': 42
        }

        clf = XGBClassifier(**xgb_params)

        mean_accuracy = InnerCrossValidation(clf, self.X, self.y, cv_method, n_fold)

        return mean_accuracy

In [ ]:
clf = XGBClassifier(random_state=42)
shap_values, X_test, metric_df = NestedCVwithOptuna(XBGObjective, clf, 'xgboost')

## 40개의 모델의 개별 performance metric + 평균 performnace metric을 저장한다
## 각 모델별 test set의 feature에 따른 shap value를 저장한다

In [ ]:
if label_name == 'three_label':
    shap_values_df_0 = pd.DataFrame(shap_values[0], columns=X_test.columns)
    shap_values_df_1 = pd.DataFrame(shap_values[1], columns=X_test.columns)
    shap_values_df_2 = pd.DataFrame(shap_values[2], columns=X_test.columns)

    with pd.ExcelWriter(f"./xgb_result_2_label.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df_0.to_excel(writer, sheet_name="shap_list_0", index=False)
        shap_values_df_1.to_excel(writer, sheet_name="shap_list_1", index=False)
        shap_values_df_2.to_excel(writer, sheet_name="shap_list_2", index=False)

if label_name != 'three_label':
    shap_values_df = pd.DataFrame(shap_values, columns=X_test.columns)

    with pd.ExcelWriter(f"./xgb_result_2_label.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df.to_excel(writer, sheet_name="shap_list", index=False)

## 2 Labels

In [ ]:
if label_name != 'three_label':
    shap.summary_plot(shap_values, X_test, show=False)
    plt.savefig('shap_summaryplot_xgb_2label.png', bbox_inches='tight')

In [ ]:
if label_name != 'three_label':
    shap.summary_plot(shap_values, X_test, plot_type='bar', show=False)
    plt.savefig('shap_absmeanplot_xgb_2label.png', bbox_inches='tight')

# Random Forest Bayesian

In [ ]:
class RFObjective(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __call__(self, trial: Trial):
        rf_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'random_state': 42
        }

        clf = RandomForestClassifier(**rf_params)

        mean_accuracy = InnerCrossValidation(clf, self.X, self.y, cv_method, n_fold)

        return mean_accuracy

In [ ]:
clf = RandomForestClassifier(random_state=42)
shap_values, X_test, metric_df = NestedCVwithOptuna(RFObjective, clf, 'randomforest')

## 40개의 모델의 개별 performance metric + 평균 performnace metric을 저장한다
## 각 모델별 test set의 feature에 따른 shap value를 저장한다

In [ ]:
if label_name == 'three_label':
    shap_values_df_0 = pd.DataFrame(shap_values[0], columns=X_test.columns)
    shap_values_df_1 = pd.DataFrame(shap_values[1], columns=X_test.columns)
    shap_values_df_2 = pd.DataFrame(shap_values[2], columns=X_test.columns)

    with pd.ExcelWriter(f"./rf_result_2_label.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df_0.to_excel(writer, sheet_name="shap_list_0", index=False)
        shap_values_df_1.to_excel(writer, sheet_name="shap_list_1", index=False)
        shap_values_df_2.to_excel(writer, sheet_name="shap_list_2", index=False)

if label_name != 'three_label':
    shap_values_df = pd.DataFrame(shap_values[0], columns=X_test.columns)

    with pd.ExcelWriter(f"./rf_result_2_label.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df.to_excel(writer, sheet_name="shap_list", index=False)

## 2 Labels

In [ ]:
if label_name != 'three_label':
    shap.summary_plot(shap_values[0], X_test, show=False)
    plt.savefig('shap_summaryplot_rf_2label_class0.png', bbox_inches='tight')

In [ ]:
if label_name != 'three_label':
    shap.summary_plot(shap_values[1], X_test, show=False)
    plt.savefig('shap_summaryplot_rf_2label_class1.png', bbox_inches='tight')

In [ ]:
if label_name != 'three_label':
    shap_values = [shap_values[0], shap_values[1]]
    shap.summary_plot(shap_values, X_test, class_names=['BAL 0-0.06', 'BAL 0.06-0.09'], class_inds=[0, 1], show=False)
    plt.savefig('shap_absmeanplot_rf_2label.png', bbox_inches='tight')

# GBM Bayesian

In [ ]:
class GBMObjective(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __call__(self, trial: Trial):
        gbm_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'learning_rate': trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'max_features': trial.suggest_categorical('max_features', [None, 'sqrt']),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.1), 
            'random_state': 42
        }

        clf = GradientBoostingClassifier(**gbm_params)

        mean_accuracy = InnerCrossValidation(clf, self.X, self.y, cv_method, n_fold)

        return mean_accuracy

In [ ]:
clf = GradientBoostingClassifier(random_state=42)
shap_values, X_test, metric_df = NestedCVwithOptuna(GBMObjective, clf, 'gbm')

## 40개의 모델의 개별 performance metric + 평균 performnace metric을 저장한다
## 각 모델별 test set의 feature에 따른 shap value를 저장한다

In [ ]:
if label_name == 'three_label':
    shap_values_df_0 = pd.DataFrame(shap_values[0], columns=X_test.columns)
    shap_values_df_1 = pd.DataFrame(shap_values[1], columns=X_test.columns)
    shap_values_df_2 = pd.DataFrame(shap_values[2], columns=X_test.columns)

    with pd.ExcelWriter(f"./gbm_result_2_label.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df_0.to_excel(writer, sheet_name="shap_list_0", index=False)
        shap_values_df_1.to_excel(writer, sheet_name="shap_list_1", index=False)
        shap_values_df_2.to_excel(writer, sheet_name="shap_list_2", index=False)

if label_name != 'three_label':
    shap_values_df = pd.DataFrame(shap_values, columns=X_test.columns)

    with pd.ExcelWriter(f"./gbm_result_2_label.xlsx") as writer:
        metric_df.to_excel(writer, sheet_name="performance metric", index=False)
        shap_values_df.to_excel(writer, sheet_name="shap_list", index=False)

## 2 Labels

In [ ]:
if label_name != 'three_label':
    shap.summary_plot(shap_values, X_test, show=False)
    plt.savefig('shap_summaryplot_gbm_2label.png', bbox_inches='tight')

In [ ]:
if label_name != 'three_label':
    shap.summary_plot(shap_values, X_test, plot_type='bar', show=False)
    plt.savefig('shap_absmeanplot_gbm_2label.png', bbox_inches='tight')